# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,834.943503,-0.672054,1234,834.271449,835.615556,1668.542898,1671.231113
6,474.078556,0.219877,1234,474.298433,473.858678,948.596866,947.717356
7,709.519057,-0.266908,1234,709.252148,709.785965,1418.504297,1419.571930
8,18.415548,0.895726,1234,19.311274,17.519821,38.622548,35.039642
9,202.144895,0.646535,1234,202.791430,201.498360,405.582860,402.996721
10,911.998281,0.807516,1234,912.805797,911.190765,1825.611594,1822.381530
11,75.678136,0.085346,1234,75.763481,75.592790,151.526963,151.185581
12,88.921529,-0.872417,1234,88.049113,89.793946,176.098225,179.587892
13,269.241116,0.684627,1234,269.925743,268.556488,539.851486,537.112977
14,30.932402,0.683752,1234,31.616154,30.248649,63.232308,60.497298


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-834.271449,835.615556
6,-474.298433,473.858678
7,-709.252148,709.785965
8,-19.311274,17.519821
9,-202.791430,201.498360
10,-912.805797,911.190765
11,-75.763481,75.592790
12,-88.049113,89.793946
13,-269.925743,268.556488
14,-31.616154,30.248649


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-834.271449,835.615556
6,-474.298433,473.858678
7,-709.252148,709.785965
8,-19.311274,17.519821
9,-202.791430,201.498360
10,-912.805797,911.190765
11,-75.763481,75.592790
12,-88.049113,89.793946
13,-269.925743,268.556488
14,-31.616154,30.248649


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-834.27144878,  835.61555634],
       [-474.29843321,  473.85867824],
       [-709.25214841,  709.78596518],
       [ -19.31127383,   17.51982123],
       [-202.79142996,  201.49836046],
       [-912.80579697,  911.19076484],
       [ -75.76348145,   75.59279044],
       [ -88.04911256,   89.79394584],
       [-269.92574282,  268.55648845],
       [ -31.61615405,   30.24864914]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1666.542898,-1671.231113
6,946.596866,-947.717356
7,1416.504297,-1419.571930
8,36.622548,-35.039642
9,403.582860,-402.996721
10,1823.611594,-1822.381530
11,149.526963,-151.185581
12,174.098225,-179.587892
13,537.851486,-537.112977
14,61.232308,-60.497298


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
